# Technical Notebook
#### Authors: Ian Reinert and Peter Haferl
#### Date: 25/09/2019
#### Data source: https://www.bjs.gov/content/data/ncrp/2004/da26521-0003.zip

In [1]:
from clean_explore import *

In [2]:
data = pd.read_csv('data/da26521-0003.tsv', sep='\t', header=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Report - Predicting Parole Completion

<em> Results: ~80% success rate in predicting parole completion in 2004 parole release data </em>
#### Most important features: 

### Assumptions
- Removing missing values (roughly 20% of the data) will not excessively bias results.
- It is valid to drop columns with missing value share greater than 0.20
- Colinearity between remaining time-related features will not excessively bias results
- Measurement error does not excessively bias results (data collected via filling out forms).

### Open issues/problems

- Trying to minimize implicit racial and gender bias.
- Possible leakage in data cleaning (data was not split before clean).
- More rigorous SVM optimization

### Analysis

<b>Our steps were as follows:</b><br>
<b> 1. Clean and explore data</b> - notable steps were replacing life sentences (coded '99999') with the 2004 average life sentence of 29 years (https://www.opensocietyfoundations.org/publications/meaning-life-long-prison-sentences-context), labeling our outcome variable as either 1 (parole completed) or 0 (state supervision continues) dropping variables that were contingent on our outcome variable (e.g. time served on parole, age at parole release), and removing gender and race variables.<br>
<b> 2. Running a baseline logistic regression model</b>: <em>Accuracy: 70% </em><br>
<b> 3. Building optimized models</b>: <em> Best Optimized Model (random forest) Accuracy: 80%